In [3]:
import tweepy
from IPython.display import clear_output
import json
import pandas as pd
from time import sleep
import datetime
import os
from textblob import TextBlob
#import twitter apps configaration file from config.py file
from config import keys as k
Players=["GamsuOW","STRIKER_OW","blaseOW","aimgod_ow","KellexOW","HarryHook","MickiePP",
         "DF_aKm","uNKOE","OGE_ow","HAGOPEUN_OW","minon501","MisfitsTviQuE","xepher123",
         "Apply","Muma","BaniOW","Forceboink","LiNkzrOW","coolmatt69","Birdring","BdosinOW",
         "NUS_OW","Furyy_d", "Profit_OWL","Surefour", "ow_void","BischuGG", "Shaz_OW","Hydration",
         "karivow","spaceOW","Custa","agilities","Fate_ow","saebyeolbe","MekOppa","Anamo_ow","tf2pine",
         "Mano_OWL","Boombox","poko_ow","carpe_ow","snillo_ow","SADO_OW","smurf_ow","moth_ow","Shock_CHB",
         "super_OW","KING_BABYBAY"]


In [6]:
class tweets():
    def __init__(self,keys):
        #Authentication using keys & accesstoken
        auth = tweepy.OAuthHandler(keys.consumer_key, keys.consumer_secret)
        auth.set_access_token(keys.access_token, keys.access_token_secret)
        self.api = tweepy.API(auth)
    def get_info_about_user(self,user_name):
        #Donal Trump Twitter account
        user = self.api.get_user(user_name)
        print("Name:", user.name)
        print("User id: ", user.id_str)
        print("Description: ", user.description)
        print("Location:",user.location)
        print("Time zone: ", user.time_zone)
        print("Number of Following:",user.friends_count)
        print("Number of Followers:",user.followers_count)
        print("Number of tweets: ", str(user.statuses_count))
    def get_realtions(self,user_id,amount):
        df= pd.DataFrame(columns=['from','to','created_at'])
        tmplst=[]
        for user in tweepy.Cursor(self.api.followers, screen_name=user_id, count=amount).items():
            sleep(1)
            clear_output()
            tmplst.append({ 'from':user.screen_name,
                            'to':user_id,
                            'created_at':user.created_at})
            print('Followers:',(round(((len(tmplst)/amount)*100),2)),'%')
        for user in tweepy.Cursor(self.api.friends, screen_name=user_id,count=amount).items():
            sleep(1)
            clear_output()
            tmplst.append({ 'from':user_id,
                            'to':user.screen_name,
                            'created_at':user.created_at})
            print('Friends:'(round(((len(tmplst)/amount)*100),2)),'%')
            
        df= pd.DataFrame(tmplst)
        return df

In [7]:
t=tweets(k)
# tmplist=[]
# for name in Players:
#     print(name)
#     tempdf=t.get_realtions(name)
#     templst.append(tempdf)
df=t.get_realtions("GamsuOW",100)

    

RateLimitError: [{'message': 'Rate limit exceeded', 'code': 88}]

In [ ]:
df